# Kipr personal toolbox speed checks


In [1]:
import kipr as kp
import numpy as np

In [3]:
shape = (10, 10)
a, b = kp.arr(1.14, shape=shape), kp.arr(1.542, shape=shape)
%timeit a + b

a, b = np.ones(shape, dtype=np.float32), np.ones(shape, dtype=np.float32)
%timeit a + b

274 ns ± 38.7 ns per loop (mean ± std. dev. of 7 runs, 1000000 loops each)
577 ns ± 49.7 ns per loop (mean ± std. dev. of 7 runs, 1000000 loops each)


In [4]:
shape = (10, 10, 10, 10, 10, 10)
values = np.random.randn(*shape)
values2 = np.random.randn(10000, 300, 10)

a = kp.arr(values)
b = kp.arr(values2)
print('kipr time:')
%timeit a[0], a[3, 2, 5, 2, 5, 2], a[:, ..., 1:7:2], a[[3, 2, 5, 2, 5, 2], :, 0], b[3, 2, 5], b[:, ..., 1:7:2]

a = values
b = values2
print('numpy time:')
%timeit a[0].copy(), a[3, 2, 5, 2, 5, 2].copy(), a[:, ..., 1:7:2].copy(), a[[3, 2, 5, 2, 5, 2], :, 0], b[3, 2, 5].copy(), b[:, ..., 1:7:2]

kipr time:
167 ms ± 30.2 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
numpy time:
2.03 ms ± 70.6 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


In [6]:
shape = (10, 10, 10, 10, 10, 10)
values = np.random.randn(*shape)
a = kp.arr(values)

In [2]:
a[0]

NameError: name 'a' is not defined

In [9]:
import os
os.getpid()

3928